In [2]:
import concurrent.futures
import os
import numpy as np
import pickle
import time

In [54]:
FILEPATHS = []
root_dir="/home/project_x/data/"
scope = 'training'
root_path = root_dir + scope
for cam in ['SIDE_RIGHT']:
    cam_filepaths = os.listdir(f"{root_path}/{cam}")
    FILEPATHS += [f"{root_path}/{cam}/{i}" for i in cam_filepaths]

In [55]:
len(FILEPATHS)

158081

In [56]:
FILEPATHS[39986]

'/home/project_x/data/training/SIDE_RIGHT/blob_236_frame_50.pickle'

In [57]:
def load_pickle(fpath):
    with open(fpath, "rb") as f:
        return pickle.load(f)

In [58]:
def load_annotations(annot):
        # get ground truth annotations
        annotations = np.zeros((0, 5))

        # some images appear to miss annotations
        if len(annot) == 0:
            return annotations

        # parse annotations
        for idx, a in enumerate(annot):

            # some annotations have basically no width / height, skip them
            if a.box.width < 1 or a.box.length < 1:
                continue

            annotation = np.zeros((1, 5))
            annotation[0, :4] = [
            a.box.center_x -0.5*a.box.length,
            a.box.center_y -0.5*a.box.width,
            a.box.center_x +0.5*a.box.length,
            a.box.center_y +0.5*a.box.width,
            ]
            annotation[0, 4] = a.type-1
            annotations = np.append(annotations, annotation, axis=0)


        return annotations

In [59]:
def ludos_help(fpath):
    try:
        return load_annotations(load_pickle(fpath)['labels'].labels).any()
    except EOFError:
        print('Pickle failed on ', fpath)
        return False

In [60]:
def get_paths(ran, filepaths=FILEPATHS):
    fro, up_to = ran
    filepaths = [
        i for i in filepaths[fro:up_to] if ludos_help(i)
    ]
    return filepaths

In [61]:
if scope == 'validation':
    start = time.perf_counter()
    with concurrent.futures.ThreadPoolExecutor(max_workers = 8) as executor:
        args1 = range(0, 39000, 1000)
        args2 = range(1000, 40000, 1000)
        args = list(zip(args1, args2))
        args3 = [(39000, 39987)]
        args += args3
        fs = executor.map(get_paths, args)
    finish = time.perf_counter()
    print(f'{finish - start}')

elif scope == 'training':
    start = time.perf_counter()
    with concurrent.futures.ThreadPoolExecutor(max_workers = 8) as executor:
        args1 = range(0, 158000, 1000)
        args2 = range(1000, 159000, 1000)
        args = list(zip(args1, args2))
        args3 = [(158000, 158081)]
        args += args3
        fs = executor.map(get_paths, args)
    finish = time.perf_counter()
    print(f'{finish - start}')

1359.7997817269643


In [62]:
whole = []
for file_p in fs:
    whole += file_p

In [63]:
with open(f'{scope}_{cam}_with_objects.pickle', "wb") as outfile:
    pickle.dump(whole, outfile)